# Chapter 7: NLP LSTM
## Ex2: IMDB Sentiment Classification with LSTM - Keras
* Cho dữ liệu https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz (https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz) (có sẵn trong from tensorflow.keras.datasets import imdb)
* Xây dựng model dự đoán một nội dung là 1 hay là 0

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LDS8_K275_ONLINE_NGUYENTHIKIMHOANG/Week_4/Chapter7

/content/drive/MyDrive/LDS8_K275_ONLINE_NGUYENTHIKIMHOANG/Week_4/Chapter7


In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
%matplotlib inline

In [ ]:
max_features = 20000
maxlen = 80 # cut texts after this number of words (among top max_features most 
batch_size = 32

In [ ]:
print('Loading data...')
(x_train_original, y_train), (x_test_original, y_test) = imdb.load_data(num_words = max_features)
print(len(x_train_original), 'train sequences')
print(len(x_test_original), 'test sequences')

Loading data...
17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [ ]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train_original, maxlen=maxlen) 
x_test = sequence.pad_sequences(x_test_original, maxlen=maxlen) 
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128)) 
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

Build model...


In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                

In [ ]:
print('Train...')
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    validation_data=(x_test, y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Train...
Epoch 1/15
782/782 [==============================] - 261s 330ms/step - loss: 0.4368 - accuracy: 0.7930 - val_loss: 0.3765 - val_accuracy: 0.8370
Epoch 2/15
782/782 [==============================] - 257s 329ms/step - loss: 0.2594 - accuracy: 0.8962 - val_loss: 0.3726 - val_accuracy: 0.8400
Epoch 3/15
782/782 [==============================] - 265s 338ms/step - loss: 0.1607 - accuracy: 0.9395 - val_loss: 0.4292 - val_accuracy: 0.8339
Epoch 4/15
782/782 [==============================] - 257s 329ms/step - loss: 0.1018 - accuracy: 0.9626 - val_loss: 0.6450 - val_accuracy: 0.8242
Epoch 5/15
782/782 [==============================] - 257s 329ms/step - loss: 0.0743 - accuracy: 0.9739 - val_loss: 0.7932 - val_accuracy: 0.8271


In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
from tensorflow.keras.utils import plot_model 
from IPython.display import Image

In [ ]:
plot_model(model, to_file='LSTM_model_Sentiment.png', show_shapes=True) 
Image(filename='LSTM_model_Sentiment.png')

In [ ]:
#Save the result
from tensorflow.keras.models import load_model 
# Creates a HDF5 file 'my_model.h5' 
model.save('LSTM_sentiment_model.h5')